In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from tqdm import tqdm
import time
import os
import re
import datetime

In [24]:
def getTables(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    html_string = str(bs)
    html_io = StringIO(html_string)
    tables = pd.read_html(html_io)
    df = tables[0]
    return bs, df

In [25]:
url = ('https://db.netkeiba.com/race/202406030811/')

url2 = ('https://race.netkeiba.com/race/result.html?race_id=202406030811&rf=race_list') 



In [26]:
colName = ['raceId', '着順', '枠番', '馬番', '馬名', '馬ID', '性別','年齢', '斤量', '騎手','騎手ID', 'タイム', '着差', '人気', '単勝オッズ',\
           '第一コーナー', '第二コーナー', '第三コーナー', '第四コーナー', '東西', '調教師', '調教師ID', '馬体重', '馬体重(前走)', '増減']
dataFrame = pd.DataFrame(columns=colName)
print(dataFrame.columns)

Index(['raceId', '着順', '枠番', '馬番', '馬名', '馬ID', '性別', '年齢', '斤量', '騎手', '騎手ID',
       'タイム', '着差', '人気', '単勝オッズ', '第一コーナー', '第二コーナー', '第三コーナー', '第四コーナー',
       '東西', '調教師', '調教師ID', '馬体重', '馬体重(前走)', '増減'],
      dtype='object')


In [27]:

bs, df = getTables(url)
time.sleep(1)
_, df2 = getTables(url2)
prehourse_goal_time =0
nowhourse_goal_time = 0
testFrame = pd.DataFrame(columns=colName)
for i in range(0, len(df)-1):
    new_row = {}
    new_row['raceId'] = '202405020801'#後で変更
    new_row['着順'] = df['着 順'][i]
    new_row['枠番'] = df['枠 番'][i]
    new_row['馬番'] = df['馬 番'][i]
    new_row['馬名'] = df['馬名'][i]
    horseName = df['馬名'][i]
    link = bs.find('a', {'title': horseName})['href']
    horse_id = link.split('/')[-2]
    new_row['馬ID'] = horse_id
    gender = df['性齢'].map(lambda x: str(x)[0])
    age = df['性齢'].map(lambda x: str(x)[1:])
    new_row['性別'] = gender[i]
    new_row['年齢'] = age[i]
    new_row['斤量'] = df['斤量'][i]
    new_row['騎手'] = df['騎手'][i]#騎手にマークが付いていることがある
    jockeyName = df['騎手'][i]
    link = bs.find('a', {'title': jockeyName})['href']
    jockey_id = link.split('/')[-2]
    new_row['騎手ID'] = jockey_id
    goaltime_min = df['タイム'].map(lambda x: str(x)[0])
    goaltime_min = int(goaltime_min[i])*60
    goaltime_second = df['タイム'].map(lambda x: str(x)[2:])
    goaltime_second = float(goaltime_second[i])
    nowhourse_goal_time = goaltime_min + goaltime_second
    new_row['タイム'] = nowhourse_goal_time
    new_row['着差'] = round(nowhourse_goal_time - prehourse_goal_time,1) if prehourse_goal_time != 0 else 0
    prehourse_goal_time = nowhourse_goal_time
    new_row['人気'] = df['人 気'][i]
    new_row['単勝オッズ'] = float(df['単勝'][i])
    corner = str(df2['コーナー 通過順'][i]).split('-')
    new_row['第一コーナー'] = corner[-4] if len(corner) >= 4 else ''
    new_row['第二コーナー'] = corner[-3] if len(corner) >= 3 else ''
    new_row['第三コーナー'] = corner[-2] if len(corner) >= 2 else ''
    new_row['第四コーナー'] = corner[-1] if len(corner) >= 1 else ''
    new_row['東西'] = df['調教師'][i][:3].replace('[','').replace(']','')
    new_row['調教師'] = df['調教師'][i][4:]
    trainerName = df['調教師'][i][4:]
    link = bs.find('a', {'title': trainerName})['href']
    trainer_id = link.split('/')[-2]
    new_row['調教師ID'] = trainer_id
    hourseWeight =  int(df['馬体重'][i][:3])
    new_row['馬体重'] = hourseWeight
    weight_change = df['馬体重'][i][3:].replace('(','').replace(')','')
    weight_change = int(weight_change)
    new_row['馬体重(前走)'] = hourseWeight - weight_change
    new_row['増減'] = weight_change

    #後で修正
    testFrame = pd.concat((testFrame,pd.DataFrame([new_row])), ignore_index=True)
print(testFrame)
testFrame.to_pickle('test.pkl')

/horse/2021105795/
/horse/2021106787/
/horse/2021107171/
/horse/2021105436/
/horse/2021110048/
/horse/2021105898/
/horse/2021101436/
/horse/2021105040/
/horse/2021104756/
/horse/2021105856/
/horse/2021105623/
/horse/2021105560/
/horse/2021105587/
/horse/2021104094/
/horse/2021105625/
/horse/2021100953/
/horse/2021103272/
          raceId  着順 枠番  馬番         馬名         馬ID 性別 年齢  斤量    騎手  ...  \
0   202405020801   1  7  13  ジャスティンミラノ  2021105795  牡  3  57  戸崎圭太  ...   
1   202405020801   2  6  12   コスモキュランダ  2021106787  牡  3  57  モレイラ  ...   
2   202405020801   3  4   8  ジャンタルマンタル  2021107171  牡  3  57  川田将雅  ...   
3   202405020801   4  5   9    アーバンシック  2021105436  牡  3  57  横山武史  ...   
4   202405020801   5  7  14    シンエンペラー  2021110048  牡  3  57  坂井瑠星  ...   
5   202405020801   6  5  10      レガレイラ  2021105898  牝  3  55  北村宏司  ...   
6   202405020801   7  2   3    エコロヴァルツ  2021101436  牡  3  57    武豊  ...   
7   202405020801   8  4   7  ルカランフィースト  2021105040  牡  3  57  松山弘平  ...   
8 

C:\Users\shirase\AppData\Local\Temp\ipykernel_11180\864166003.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testFrame = pd.concat((testFrame,pd.DataFrame([new_row])), ignore_index=True)


In [16]:
df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,13,ジャスティンミラノ,牡3,57,戸崎圭太,1:57.1,NaN,4.8,2.0,512(+10),[西] 友道康夫
1,2,6,12,コスモキュランダ,牡3,57,モレイラ,1:57.1,クビ,15.8,7.0,500(-4),[東] 加藤士津
2,3,4,8,ジャンタルマンタル,牡3,57,川田将雅,1:57.2,1/2,6.1,3.0,490(-2),[西] 高野友和
3,4,5,9,アーバンシック,牡3,57,横山武史,1:57.5,1.1/2,14.2,6.0,510(-6),[東] 武井亮
4,5,7,14,シンエンペラー,牡3,57,坂井瑠星,1:57.5,クビ,8.4,5.0,482(+2),[西] 矢作芳人
5,6,5,10,レガレイラ,牝3,55,北村宏司,1:57.6,クビ,3.7,1.0,456(+2),[東] 木村哲也
6,7,2,3,エコロヴァルツ,牡3,57,武豊,1:57.8,1.1/2,40.5,11.0,486(-10),[西] 牧浦充徳
7,8,4,7,ルカランフィースト,牡3,57,松山弘平,1:57.9,3/4,272.7,14.0,468(+6),[東] 鹿戸雄一
8,9,1,1,サンライズジパング,牡3,57,菅原明良,1:58.0,クビ,27.5,10.0,510(-6),[西] 音無秀孝
9,10,3,5,ミスタージーティー,牡3,57,藤岡佑介,1:58.1,1/2,17.4,9.0,468(-4),[西] 矢作芳人


In [ ]:
raceResultDf = pd.read_pickle('test.pkl')
raceResultDf

In [ ]:
#後で消す
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.status_code == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response.content, 'html.parser')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)
        return df

    else:
        print(f"HTTPステータスコード {response.status_code}: ページの取得に失敗しました")
        

In [ ]:
#後で消す
table = getTables(response)
rows = table.find_all('tr')
rows['枠']

In [ ]:
#開催年
years = [str(i).zfill(4) for i in range(2021, 2022)]
#開催場所 01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09::阪神, 10:小倉
places = [str(i).zfill(2) for i in range (5, 6)]
#開催回
times = [str(i).zfill(2) for i in range(1, 2)]
#開催日
days = [str(i).zfill(2) for i in range(1, 2)]
#レースNo
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in places:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)
                    

In [ ]:

url = 'https://db.netkeiba.com/race/'
colName = ['raceId', 'htmlBytes']
df = pd.DataFrame(columns=colName)
escapeList = []

# pickleファイルが存在するか確認し、データを読み込む
if os.path.isfile('race_html.pkl'):
    df = pd.read_pickle('race_html.pkl')
    escapeList = df['raceId'].to_list()

# ページネーション用の競走IDリストを生成する関数
def addEscapeList(id: str, ll: list):
    idAry = [id[0:4], id[4:6], id[6:8], id[8:10], id[10:12]]
    for r in range(1, 13):
        idAry[4] = str(r).zfill(2)
        ll.append(''.join(idAry))
    if idAry[3] == '01':
        for d in range(2, 9):
            idAry[3] = str(d).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)
    if idAry[2] == '01':
        for t in range(2, 9):
            idAry[2] = str(t).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)

    return ll

# raceIdListに実際の競走IDのリストがあると仮定します
# raceIdList = ...

for raceId in tqdm(raceIdList):
    try:
        if raceId in escapeList:
            continue
        response = url + raceId
        html = requests.get(response)
        #html.encoding = 'utf-8'
        soup = BeautifulSoup(html.content, 'html.parser')

        if 'レース結果' in soup.text:
            tmpDf = pd.DataFrame([[raceId, html.content]], columns=colName)  # 'colmuns'の誤りを修正
            df = pd.concat([df, tmpDf], axis=0, ignore_index=True)
        else:
            escapeList = addEscapeList(raceId, escapeList)  # 変数名を修正

        time.sleep(3)
    except Exception as e:  # 例外をキャッチしてログに記録
        print(f'例外が発生しました: {str(e)}')

# DataFrameをpickleファイルに保存
df.to_pickle('race_html.pkl')


In [ ]:
htmlDf = pd.read_pickle('race_html.pkl')

raceInfoList = []

#raceResultDf = pd.DataFrame()
for idx, dat in tqdm(htmlDf.iterrows(), total=len(htmlDf)):
    raceId = dat['raceId']
    htmlBytes = dat['htmlBytes']

    soup = BeautifulSoup(htmlBytes.decode('euc-jp','ignore'), 'html.parser')
    mainrace_data = soup.find('div', class_='mainrace_data')
    rowdata = {}
    rowdata['raceId'] = raceId
    rowdata['レース名'] = mainrace_data.find('h1').text
    rowdata['R'] = mainrace_data.find('dt').text.replace('\n','').replace(' ','').replace('R', '')
    spantexts = mainrace_data.find('span').text.replace('\xa0','').replace(' ','').split('/')
    
    rowdata['コース種'] = '障害' if '障' in spantexts[0] else 'ダート' if 'ダ' in spantexts[0] else '芝'
    
    rowdata['コース回り'] = '右' if '右' in spantexts[0] else '左' if '左' in spantexts[0] else '障害'

    rowdata['距離'] = int(re.findall('\d+',spantexts[0])[0])

    rowdata['天気'] = spantexts[1][3:]

    rowdata['馬場'] = spantexts[2].split(':')[1]
    rowdata['発走'] = spantexts[3][3:]

    smalltxt = mainrace_data.find('p', class_='smalltxt').text.replace('\xa0',' ').replace('  ',' ').split(' ')
    dt = datetime.datetime.strptime(smalltxt[0], '%Y年%m月%d日')
    rowdata['日付'] = dt.strftime('%Y/%m/%d')

    placeDict = {
        '01':'札幌', '02':'函館', '03':'福島', '04':'新潟', '05':'東京', '06':'中山', '07':'中京', '08':'京都', '09':'阪神', '10':'小倉'
    }
    rowdata['開催場所'] = placeDict[raceId[4:6]]

    if 'G1' in rowdata['レース名']:
        raceGrade = 'G1'
    elif 'G2' in rowdata['レース名']:
        raceGrade = 'G2'
    elif 'G3' in rowdata['レース名']:
        raceGrade = 'G3'
    elif '未勝利' in smalltxt[2]:
        raceGrade = '未勝利'
    elif '新馬' in smalltxt[2]:
        raceGrade = '新馬'
    elif '1勝クラス' in smalltxt[2] or '500万' in smalltxt[2]:
        raceGrade = '1勝クラス'
    elif '2勝' in smalltxt[2] or '1000万' in smalltxt[2]:
        raceGrade = '2勝クラス'
    elif '3勝' in smalltxt[2] or '1600万' in smalltxt[2]:   
        raceGrade = '3勝クラス'
    else:
        raceGrade = 'オープン'  

    if '牡・牝' in smalltxt[3]:
        restriction = '牡・牝'
    elif '牝' in smalltxt[3]:
        restriction = '牝'
    elif '牡' in smalltxt[3]:
        restriction = '牡'
    else:
        restriction = '無'

    rowdata['制限'] = restriction

    if 'ハンデ' in smalltxt[3]:
        handicap = 'ハンデ'
    elif '別定' in smalltxt[3]:
        handicap = '別定'
    else:
        handicap = '定量'
    
    rowdata['ハンデ'] = handicap

    raceInfoList.append(rowdata)
raceInfoDf = pd.DataFrame(raceInfoList)
raceInfoDf.to_pickle('race_info.pkl')

In [ ]:
raceResultDf = pd.read_pickle('race_info.pkl')
raceResultDf

In [ ]:
table = soup.find_all('table')[0]

columns = []

for head in table.find_all('th'):
    columns.append(head.text)

columns = ['raceId'] + columns + ['horseId', 'jockeyId', 'trainerId']
df = pd.DataFrame(columns=columns)

for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        continue
    items = [raceId]

    cells = row.find_all('td')

    for cell in cells:
        items.append(cell.text.replace('\n',''))

    items.append(str(cells[3]).split('/horse/')[1].split('/')[0])
    items.append(str(cells[6]).split('/recent/')[1].split('/')[0])
    items.append(str(cells[18]).split('/recent')[1].split('/')[0])

    df.loc[i] = items
    break

raceResultDf = pd.concat([raceResultDf, df],axis=0)


In [ ]:
htmlBytes

In [ ]:
df.to_csv('data.csv',encoding='utf-8')
df.columns
print(df)